In [34]:
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.express as px

DATA = Path("data/processed/analytics_table.parquet")
FIGS = Path("reports/figures")
FIGS.mkdir(parents=True, exist_ok=True)

def save_fig(fig, path: Path, *, scale: int = 2) -> None:
    """Save a Plotly figure to disk (requires `kaleido`)."""
    path.parent.mkdir(parents=True, exist_ok=True)
    fig.write_image(str(path), scale=scale)

In [35]:
DATA = Path("../data/processed/analytics_table.parquet")
FIGS = Path("../reports/figures")
FIGS.mkdir(parents=True, exist_ok=True)

def save_fig(fig, path: Path, *, scale: int = 2) -> None:
    """Save a Plotly figure to disk (requires `kaleido`)."""
    path.parent.mkdir(parents=True, exist_ok=True)
    fig.write_image(str(path), scale=scale)

In [36]:
df = pd.read_parquet(DATA)

print("rows:", len(df), "cols:", len(df.columns))
print(df.dtypes.head(15))

missing = df.isna().sum().sort_values(ascending=False).head(10)
print(missing)

rows: 5 cols: 18
order_id               string[python]
user_id                string[python]
amount                        Float64
quantity                        Int64
created_at        datetime64[ns, UTC]
status                         object
status_clean                   object
amount__isna                     bool
quantity__isna                   bool
date                           object
year                          float64
month                  string[python]
dow                            object
hour                          float64
country                        object
dtype: object
quantity              1
amount                1
amount__is_outlier    1
amount_winsor         1
created_at            1
date                  1
hour                  1
dow                   1
month                 1
year                  1
dtype: int64


In [52]:

print("=== DATA REALITY CHECK ===")
print(f"Total orders: {len(df)}")
print(f"Date range: {df['created_at'].min()} to {df['created_at'].max()}")


=== DATA REALITY CHECK ===
Total orders: 5
Date range: 2025-12-01 10:05:00+00:00 to 2025-12-03 14:30:00+00:00


In [37]:
print("\nColumn names:")
print(list(df.columns))

print("\nFirst few rows:")
print(df.head(3))


Column names:
['order_id', 'user_id', 'amount', 'quantity', 'created_at', 'status', 'status_clean', 'amount__isna', 'quantity__isna', 'date', 'year', 'month', 'dow', 'hour', 'country', 'signup_date', 'amount_winsor', 'amount__is_outlier']

First few rows:
  order_id user_id  amount  quantity                created_at  status  \
0    A0001    0001    12.5         1 2025-12-01 10:05:00+00:00    Paid   
1    A0002    0002     8.0         2 2025-12-01 11:10:00+00:00    paid   
2    A0003    0003    <NA>         1 2025-12-02 09:00:00+00:00  Refund   

  status_clean  amount__isna  quantity__isna        date    year    month  \
0         paid         False           False  2025-12-01  2025.0  2025-12   
1         paid         False           False  2025-12-01  2025.0  2025-12   
2       refund          True           False  2025-12-02  2025.0  2025-12   

       dow  hour country signup_date  amount_winsor  amount__is_outlier  
0   Monday  10.0      SA  2025-11-15           12.5            

## EDA Questions

1. **Trend Question**: How does total revenue change by month?
2. **Segment Question**: What is the refund rate by country (with n orders)?
3. **Distribution Question**: What is the distribution of order amounts?
4. **Data Quality Question**: How complete are our key columns (country, status, amount)?

In [45]:

if "month" not in df.columns and "created_at" in df.columns:
    dt = pd.to_datetime(df["created_at"], errors="coerce", utc=True)
    df["month"] = dt.dt.to_period("M").astype("string")
    print("Created 'month' column")

In [46]:

if "amount_winsor" not in df.columns and "amount" in df.columns:
    lo, hi = df["amount"].quantile([0.01, 0.99])
    df["amount_winsor"] = df["amount"].clip(lo, hi)
    print(f"Created 'amount_winsor' (clipped at {lo:.2f}, {hi:.2f})")

In [40]:
# Question 1: Revenue by country
rev_by_country = (
    df.groupby("country", dropna=False)
    .agg(
        n=("order_id", "size"),
        revenue=("amount", "sum"),
        aov=("amount", "mean")  # Average Order Value
    )
    .reset_index()
    .sort_values("revenue", ascending=False)
)

print("Revenue by Country:")
print(rev_by_country)

# Create chart
fig = px.bar(rev_by_country, x="country", y="revenue", 
             title="Revenue by Country (All Time)")
fig.update_layout(title={"x": 0.02})
fig.update_xaxes(title_text="Country")
fig.update_yaxes(title_text="Revenue (sum of amount)")
fig.show()

# Export
save_fig(fig, FIGS / "revenue_by_country.png")
print("Exported to reports/figures/revenue_by_country.png")

Revenue by Country:
  country  n  revenue     aov
1      SA  4    145.5  36.375
0      AE  1      0.0    <NA>


Exported to reports/figures/revenue_by_country.png


In [47]:
# Question 2: Revenue trend monthly
revenue_trend = (
    df.groupby("month", dropna=False)
    .agg(
        n=("order_id", "size"),
        revenue=("amount", "sum")
    )
    .reset_index()
    .sort_values("month")  # Important: sort by time!
)

print("Revenue Trend by Month:")
print(revenue_trend)

# Create line chart
fig = px.line(revenue_trend, x="month", y="revenue", 
             title="Revenue Over Time (Monthly)")
fig.update_layout(title={"x": 0.02})
fig.update_xaxes(title_text="Month")
fig.update_yaxes(title_text="Revenue")
fig.show()

# Export
save_fig(fig, FIGS / "revenue_trend_monthly.png")
print("Exported to reports/figures/revenue_trend_monthly.png")

Revenue Trend by Month:
     month  n  revenue
0  2025-12  4     45.5
1     <NA>  1    100.0


Exported to reports/figures/revenue_trend_monthly.png


In [48]:
# Question 3: Amount distribution
print("Amount statistics:")
print(df["amount"].describe())

# Histogram of winsorized amount
fig = px.histogram(df, x="amount_winsor", nbins=30,
                   title="Order Amount Distribution (Winsorized)")
fig.update_layout(title={"x": 0.02})
fig.update_xaxes(title_text="Amount (winsorized)")
fig.update_yaxes(title_text="Number of Orders")
fig.show()

# Export
save_fig(fig, FIGS / "amount_hist_winsor.png")
print("Exported to reports/figures/amount_hist_winsor.png")

Amount statistics:
count          4.0
mean        36.375
std      43.022039
min            8.0
25%         11.375
50%          18.75
75%          43.75
max          100.0
Name: amount, dtype: Float64


Exported to reports/figures/amount_hist_winsor.png
